# Tabular Playground - Jan 2021 - LightGBM

# 1. Librerias a utilizar

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import lightgbm as lgb

#Pipeline
from sklearn.pipeline import Pipeline

#Tuning de Parametros
from sklearn.model_selection import GridSearchCV

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


# 2. Dataset de entrenamiento / Dataset de testing

In [2]:
#Dataset con los datos de entrenamiento

train = pd.read_csv(input_path / 'train.csv', index_col='id')
train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [3]:
#Dataset con los datos de testing

test = pd.read_csv(input_path / 'test.csv', index_col='id')
train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [4]:
#Dataset para realizar el submit

submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

,target
id,
0,0.5
2,0.5
6,0.5
7,0.5
10,0.5


# 3. Split del dataset train

In [5]:
#Selecciono la columna del dataset que tiene la etiqueta del target

target = train['target']
target.head()

id
1    7.243043
3    8.203331
4    7.776091
5    6.957716
8    7.951046
Name: target, dtype: float64

In [6]:
#Realizo la division de train/test
#Elijo una division 75% / 25%

train_clean = train.drop(columns=['target'])
X_train, X_test, y_train, y_test = train_test_split(train_clean, target, train_size=0.75)

In [7]:
X_train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
99448,0.484465,0.422291,0.554839,0.486581,0.545308,0.480667,0.574733,0.352565,0.425524,0.520046,0.462005,0.200520,0.371652,0.601258
302857,0.360453,0.421469,0.497749,0.324254,0.283027,0.655041,0.770783,0.466791,0.496708,0.361567,0.409373,0.713472,0.677482,0.296555
499809,0.459334,0.791790,0.900919,0.317382,0.282050,0.731059,0.849220,0.370858,0.407661,0.510881,0.717925,0.711587,0.785053,0.620671
15809,0.342234,0.621695,0.735227,0.375752,0.503693,0.588811,0.488773,0.557735,0.416989,0.578614,0.753402,0.778661,0.366356,0.245954
60697,0.439333,0.063112,0.463845,0.209590,0.284121,0.334258,0.218705,0.327572,0.379139,0.204169,0.251960,0.274271,0.306475,0.814135


In [8]:
X_test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
57272,0.331303,0.619386,0.550444,0.497911,0.487656,0.333034,0.375795,0.680938,0.418207,0.342957,0.438992,0.475739,0.340246,0.268102
70862,0.817307,0.798608,0.549957,0.832304,0.285205,0.195804,0.505895,0.279390,0.342729,0.239168,0.168245,0.625371,0.331855,0.733189
167230,0.273926,0.277056,0.621358,0.318194,0.441852,0.458293,0.365506,0.355644,0.306452,0.312445,0.360837,0.296779,0.647756,0.817576
398110,0.695125,0.622358,0.617889,0.632354,0.453791,0.581153,0.898752,0.344757,0.581879,0.799825,0.816821,0.832424,0.909831,0.397050
133948,0.348834,0.357541,0.606664,0.248176,0.816755,0.425853,0.392083,0.255242,0.350307,0.516022,0.245767,0.569196,0.340257,0.832324


In [9]:
y_train.head()

id
99448     6.664828
302857    8.298143
499809    7.712710
15809     8.223338
60697     7.938620
Name: target, dtype: float64

In [10]:
y_test.head()

id
57272     8.613956
70862     8.580826
167230    7.479025
398110    7.774128
133948    8.582058
Name: target, dtype: float64

# 4. LightGBM

In [11]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'type': 'regression_l1',
    'metric': ['rmse'],
    "num_iterations": 5000,
    'verbose': 0}

In [12]:
gbm = lgb.LGBMRegressor(**hyper_params)

In [13]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=1000)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.6207	valid_0's rmse: 0.73419
Training until validation scores don't improve for 1000 rounds
[2]	valid_0's l1: 0.619169	valid_0's rmse: 0.732212
[3]	valid_0's l1: 0.617864	valid_0's rmse: 0.73062
[4]	valid_0's l1: 0.616615	valid_0's rmse: 0.729152
[5]	valid_0's l1: 0.615528	valid_0's rmse: 0.7279
[6]	valid_0's l1: 0.614471	valid_0's rmse: 0.726751
[7]	valid_0's l1: 0.613562	valid_0's rmse: 0.725821
[8]	valid_0's l1: 0.612638	valid_0's rmse: 0.724827
[9]	valid_0's l1: 0.611858	valid_0's rmse: 0.724011
[10]	valid_0's l1: 0.611083	valid_0's rmse: 0.723185
[11]	valid_0's l1: 0.610512	valid_0's rmse: 0.722573
[12]	valid_0's l1: 0.609763	valid_0's rmse: 0.721823
[13]	valid_0's l1: 0.608993	valid_0's rmse: 0.721062
[14]	valid_0's l1: 0.608402	valid_0's rmse: 0.720501
[15]	valid_0's l1: 0.607804	valid_0's rmse: 0.719894
[16]	valid_0's l1: 0.607263	valid_0's rmse: 0.719338
[17]	valid_0's l1: 0.606723	valid_0's rmse: 0.718807
[18]	valid_0's l1: 0.606214	valid_0's rmse: 0.71832

LGBMRegressor(metric=['rmse'], num_iterations=5000, objective='regression',
              task='train', type='regression_l1', verbose=0)

In [14]:
y_predicted = gbm.predict(X_test)

y_predicted

array([8.14295975, 8.06346744, 7.77018237, ..., 8.04805023, 7.76238938,
       7.98194205])

In [15]:
score_model = mean_squared_error(y_test, y_predicted, squared=False)

print(f'{score_model:0.5f}')

0.70329


# 5. Generacion del Submit

In [16]:
submission['target'] = gbm.predict(test)

submission.to_csv('gbm_regression.csv')